In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fontss-2/LondrinaSketch-Regular.ttf
/kaggle/input/fontss-2/MavenPro-Bold.ttf
/kaggle/input/fontss-2/Jura-Light.ttf
/kaggle/input/fontss-2/MavenPro-Black.ttf
/kaggle/input/fontss-2/CodaCaption-Heavy.ttf
/kaggle/input/fontss-2/JollyLodger-Regular.ttf
/kaggle/input/fontss-2/GreatVibes-Regular.ttf
/kaggle/input/fontss-2/PatrickHand-Regular.ttf
/kaggle/input/fontss-2/NoticiaText-Regular.ttf
/kaggle/input/fontss-2/ChauPhilomeneOne-Regular.ttf
/kaggle/input/fontss-2/ButchermanCaps-Regular.ttf
/kaggle/input/fontss-2/Artifika-Regular.ttf
/kaggle/input/fontss-2/AlmendraSC-Bold.ttf
/kaggle/input/fontss-2/Junge-Regular.ttf
/kaggle/input/fontss-2/MontserratSubrayada-Bold.ttf
/kaggle/input/fontss-2/PassionOne-Black.ttf
/kaggle/input/fontss-2/NanumMyeongjo-Bold.ttf
/kaggle/input/fontss-2/AveriaLibre-Light.ttf
/kaggle/input/fontss-2/Neucha.ttf
/kaggle/input/fontss-2/PTF75F.ttf
/kaggle/input/fontss-2/Ewert-Regular.ttf
/kaggle/input/fontss-2/Alegreya-BlackItalic.ttf
/kaggle/input/fontss-2/

In [2]:
pip install pillow opencv-python numpy nltk matplotlib pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision pandas pillow matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

# For reproducibility
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# ---------------------------
# PARAMETERS
# ---------------------------
# CSV_PATH = './dataset/dataset.csv'
BATCH_SIZE = 32
NUM_EPOCHS = 50
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# For classification we resize images to 64x128 (height, width) in grayscale.
IMAGE_SIZE = (64, 128)

In [5]:
import os
import random
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import nltk
from nltk.corpus import words
from matplotlib import font_manager
import pandas as pd

In [6]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from matplotlib import font_manager
import nltk
nltk.download('words', quiet=True)
from nltk.corpus import words as nltk_words

import cv2  # Needed for noise operations

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms

# Set random seeds for reproducibility.
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

##############################################
# PARAMETERS
##############################################
NUM_DISTINCT_WORDS = 100         # 100 distinct words (classes)
SAMPLES_PER_WORD = 50            # Total samples generated per word
TRAIN_SAMPLES_PER_WORD = 40      # For each word, use 40 samples for training
TEST_SAMPLES_PER_WORD = SAMPLES_PER_WORD - TRAIN_SAMPLES_PER_WORD  # (50-40 = 10)

# For image generation (Task 0 for hard set), we use images of size 128 x 64 (width, height) in RGB.
# (Note: Although the original specification mentioned 2048×64, we use a reduced resolution for demo.)
IMAGE_SIZE_TASK0 = (128, 64)

# For classification, images will be converted to grayscale and resized to 64 x 128 (height, width).
IMAGE_SIZE = (64, 128)

OUTPUT_DIR = "./dataset/hard"    # Folder where generated hard-set images will be saved
CSV_PATH = "./dataset/hard_dataset.csv"  # CSV file to store metadata

##############################################
# UTILITY FUNCTIONS FOR DATASET GENERATION
##############################################
def get_distinct_words(n, min_len=4, max_len=9):
    """Select n distinct words from NLTK corpus that are alphabetic and whose length is in [min_len, max_len]."""
    word_list = [w for w in nltk_words.words() if w.isalpha() and min_len <= len(w) <= max_len]
    word_list = list(set(word_list))
    return random.sample(word_list, n)

def add_noise_to_image(pil_image, sigma=10):
    """
    Adds Gaussian noise to the image using OpenCV.
    """
    image_np = np.array(pil_image)
    noise = np.random.normal(0, sigma, image_np.shape).astype(np.uint8)
    noisy_image = cv2.add(image_np, noise)
    return Image.fromarray(noisy_image)

def get_system_fonts():
    """Return a list of available system TrueType fonts."""
    font_paths = font_manager.findSystemFonts(fontpaths="/kaggle/input/fontss-2", fontext='ttf')
    return font_paths if font_paths else []

def create_noisy_background(image_size, background_color=None, sigma=20):
    """Creates a noisy background image with Gaussian noise."""
    background = Image.new('RGB', image_size, color=background_color)
    noisy_background = add_noise_to_image(background, sigma=sigma)
    return noisy_background

def random_capitalization(word):
    """
    Randomly change the capitalization of each letter in the word.
    For example, "hello" might become "HeLLo".
    """
    return ''.join(c.upper() if random.random() < 0.5 else c.lower() for c in word)

def generate_text_image(word, font_path, image_size=IMAGE_SIZE_TASK0, 
                        text_color=(0, 0, 0), background_color=(255, 255, 255),
                        reverse=False, use_noisy_background=False, padding=10):
    """
    Generate an image of given dimensions with the word rendered on it.
    If reverse is True, the displayed text is reversed.
    If use_noisy_background is True, the background is generated with noise.
    The function automatically adjusts the font size so that the text fits.
    """
    display_text = word[::-1] if reverse else word
    if use_noisy_background:
        image = create_noisy_background(image_size, background_color, sigma=20)
    else:
        image = Image.new('RGB', image_size, color=background_color)
    draw = ImageDraw.Draw(image)
    max_font_size = 40
    min_font_size = 10
    font_size = max_font_size
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        font = ImageFont.load_default()
    while font_size >= min_font_size:
        try:
            font = ImageFont.truetype(font_path, font_size)
        except IOError:
            font = ImageFont.load_default()
            break
        text_bbox = draw.textbbox((0, 0), display_text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        if text_width <= image_size[0] - padding and text_height <= image_size[1] - padding:
            break
        font_size -= 1
    if font_size < min_font_size:
        font_size = min_font_size
        font = ImageFont.truetype(font_path, font_size)
        text_bbox = draw.textbbox((0, 0), display_text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
    position = ((image_size[0] - text_width) // 2, (image_size[1] - text_height) // 2)
    draw.text(position, display_text, fill=text_color, font=font)
    return image

# --- For hard-set generation, we want all samples of the same word to have the same label.
# Therefore, we modify generate_hard_sample to accept a word parameter.
def generate_hard_sample(word, image_size=IMAGE_SIZE_TASK0):
    """
    For the hard set:
      - Given a word (as a string), render it with random per-letter capitalization.
      - Choose a random light background and random dark text color.
      - Use a noisy background.
    Returns a tuple (image, effective_label) where effective_label is the canonical label (in lowercase).
    """
    # For consistency, we decide that the canonical label will be word.lower().
    canonical_label = word.lower()
    # For the image appearance, we render the word with random capitalization.
    rendered_word = random_capitalization(word)
    background_color = tuple(random.randint(200, 255) for _ in range(3))
    text_color = tuple(random.randint(0, 50) for _ in range(3))
    fonts = get_system_fonts()
    if not fonts:
        raise ValueError("No system fonts found!")
    font_path = random.choice(fonts)
    img = generate_text_image(rendered_word, font_path, image_size=image_size,
                              text_color=text_color, background_color=background_color,
                              reverse=False, use_noisy_background=True)
    return img, canonical_label

def generate_hard_samples_for_word(word, num_samples=SAMPLES_PER_WORD):
    """
    For a given word, generate num_samples images for the hard set.
    Returns a list of dictionaries with keys "word" and "image_path".
    All samples for the same word will have the same canonical label (word.lower()).
    """
    samples = []
    for i in range(num_samples):
        img, effective_word = generate_hard_sample(word, image_size=IMAGE_SIZE_TASK0)
        filename = f"{effective_word}_{i}.png"
        file_path = os.path.join(OUTPUT_DIR, filename)
        img.save(file_path)
        samples.append({"word": effective_word, "image_path": file_path})
    return samples

def create_hard_classification_dataset(num_classes=NUM_DISTINCT_WORDS, samples_per_word=SAMPLES_PER_WORD):
    """
    Selects num_classes distinct words and for each word generates samples_per_word images (hard-set style).
    Saves the metadata into a CSV file and returns the list of selected words and the DataFrame.
    """
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    selected_words = get_distinct_words(num_classes)
    all_samples = []
    for word in selected_words:
        # Use the same word for all samples of this class.
        samples = generate_hard_samples_for_word(word, num_samples=samples_per_word)
        all_samples.extend(samples)
        print(f"Generated {len(samples)} samples for word: {word.lower()}")
    df = pd.DataFrame(all_samples)
    os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)
    df.to_csv(CSV_PATH, index=False)
    print(f"CSV saved at {CSV_PATH}")
    return selected_words, df

# Generate the hard-set dataset (this should create 100 * SAMPLES_PER_WORD images and a CSV file).
selected_words_hard, df_generated_hard = create_hard_classification_dataset()

##############################################
# DATASET CLASS FOR CLASSIFICATION (HARD SET)
##############################################
class OCRClassificationDataset(Dataset):
    """
    Custom dataset for word classification.
    Expects a CSV file with columns: "word" and "image_path".
    The word (label) is used to create a mapping from word to integer.
    """
    def __init__(self, csv_file, transform=None, word_to_idx=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        # Convert labels to lowercase and strip whitespace.
        self.labels = [label.strip().lower() for label in self.data["word"].tolist()]
        self.image_paths = self.data["image_path"].tolist()
        if word_to_idx is None:
            unique_words = sorted(list(set(self.labels)))
            self.word_to_idx = {word: idx for idx, word in enumerate(unique_words)}
        else:
            # Ensure keys are lowercase.
            self.word_to_idx = {word.strip().lower(): idx for word, idx in word_to_idx.items()}
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx].strip().lower()
        try:
            image = Image.open(img_path).convert("L")
        except Exception as e:
            print("Error opening image:", img_path)
            raise e
        if self.transform:
            image = self.transform(image)
        label_idx = self.word_to_idx[label]
        return image, label_idx

# Transformation for classification: resize to 64x128, convert to tensor, and normalize.
transform_class = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Build word_to_idx mapping from selected words. Here we use lowercase for consistency.
selected_words_hard_sorted = sorted([w.lower() for w in selected_words_hard])
word_to_idx = {word: idx for idx, word in enumerate(selected_words_hard_sorted)}

dataset_cls_hard = OCRClassificationDataset(CSV_PATH, transform=transform_class, word_to_idx=word_to_idx)
print(f"Total hard-set dataset size for classification: {len(dataset_cls_hard)} samples.")

##############################################
# SPLIT DATASET PER CLASS: 40 training samples and 10 test samples per class.
##############################################
from collections import defaultdict
all_indices_by_label = defaultdict(list)
for idx, (_, label_idx) in enumerate(dataset_cls_hard):
    all_indices_by_label[label_idx].append(idx)

train_indices = []
test_indices = []
for label, indices in all_indices_by_label.items():
    if len(indices) < SAMPLES_PER_WORD:
        raise ValueError(f"Not enough samples for label {label}. Expected {SAMPLES_PER_WORD}, got {len(indices)}")
    random.shuffle(indices)
    train_indices.extend(indices[:TRAIN_SAMPLES_PER_WORD])
    test_indices.extend(indices[TRAIN_SAMPLES_PER_WORD:TRAIN_SAMPLES_PER_WORD+TEST_SAMPLES_PER_WORD])

from torch.utils.data import Subset
train_subset = Subset(dataset_cls_hard, train_indices)
test_subset = Subset(dataset_cls_hard, test_indices)

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)


Generated 50 samples for word: decenyl
Generated 50 samples for word: exequy
Generated 50 samples for word: oxpecker
Generated 50 samples for word: dupedom
Generated 50 samples for word: yachan
Generated 50 samples for word: galley
Generated 50 samples for word: byrewoman
Generated 50 samples for word: binding
Generated 50 samples for word: dominican
Generated 50 samples for word: ravendom
Generated 50 samples for word: benitoite
Generated 50 samples for word: homefarer
Generated 50 samples for word: alterate
Generated 50 samples for word: savor
Generated 50 samples for word: henry
Generated 50 samples for word: unlabeled
Generated 50 samples for word: phytonomy
Generated 50 samples for word: inanimate
Generated 50 samples for word: clockroom
Generated 50 samples for word: sermoner
Generated 50 samples for word: inflect
Generated 50 samples for word: swardy
Generated 50 samples for word: bruce
Generated 50 samples for word: poultice
Generated 50 samples for word: ungrasp
Generated 50 s

In [26]:
##############################################
# MODEL DEFINITION: ResNet-18-Like Classifier
##############################################
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNetClassifier(nn.Module):
    def __init__(self, block, layers, num_classes, in_channels=1):
        super(ResNetClassifier, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(512 * block.expansion, num_classes)
    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    def forward(self, x):
        # x: (B, 1, 64, 128)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)  # Apply dropout before the fully connected layer.
        x = self.fc(x)
        return x

def ResNet18(num_classes):
    return ResNetClassifier(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

In [34]:
# Instantiate the model.
model_cls = ResNet18(num_classes=len(selected_words_hard_sorted))
model_cls = model_cls.to(DEVICE)
print(model_cls)


ResNetClassifier(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

In [35]:
criterion_cls = nn.CrossEntropyLoss()
optimizer_cls = optim.Adam(model_cls.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

def train_classifier(model, train_loader, criterion, optimizer, device, num_epochs=NUM_DISTINCT_WORDS):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    print("Training complete.")

train_classifier(model_cls, train_loader, criterion_cls, optimizer_cls, DEVICE, num_epochs=40)

def evaluate_classifier(model, test_loader, device):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f"Test Accuracy: {accuracy*100:.2f}%")
    return accuracy

evaluate_classifier(model_cls, test_loader, DEVICE)

Epoch 1/40, Loss: 4.6078
Epoch 2/40, Loss: 4.0636
Epoch 3/40, Loss: 3.6428
Epoch 4/40, Loss: 2.7233
Epoch 5/40, Loss: 1.5273
Epoch 6/40, Loss: 0.9228
Epoch 7/40, Loss: 0.6381
Epoch 8/40, Loss: 0.5090
Epoch 9/40, Loss: 0.4161
Epoch 10/40, Loss: 0.3365
Epoch 11/40, Loss: 0.3293
Epoch 12/40, Loss: 0.3023
Epoch 13/40, Loss: 0.2961
Epoch 14/40, Loss: 0.2696
Epoch 15/40, Loss: 0.2691
Epoch 16/40, Loss: 0.2911
Epoch 17/40, Loss: 0.2507
Epoch 18/40, Loss: 0.2431
Epoch 19/40, Loss: 0.2490
Epoch 20/40, Loss: 0.2682
Epoch 21/40, Loss: 0.2618
Epoch 22/40, Loss: 0.2312
Epoch 23/40, Loss: 0.2207
Epoch 24/40, Loss: 0.1971
Epoch 25/40, Loss: 0.2360
Epoch 26/40, Loss: 0.2599
Epoch 27/40, Loss: 0.2159
Epoch 28/40, Loss: 0.1969
Epoch 29/40, Loss: 0.1964
Epoch 30/40, Loss: 0.1854
Epoch 31/40, Loss: 0.1896
Epoch 32/40, Loss: 0.2107
Epoch 33/40, Loss: 0.1723
Epoch 34/40, Loss: 0.1685
Epoch 35/40, Loss: 0.1316
Epoch 36/40, Loss: 0.1227
Epoch 37/40, Loss: 0.1387
Epoch 38/40, Loss: 0.1714
Epoch 39/40, Loss: 0.

0.888

In [36]:

def evaluate_classifier(model, test_loader, device):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f"Train Accuracy: {accuracy*100:.2f}%")
    return accuracy

evaluate_classifier(model_cls, train_loader, DEVICE)

Train Accuracy: 98.28%


0.98275

In [16]:
# Save the entire model (not recommended for flexibility)
torch.save(model_cls, "model_1_hard.pth")